In [20]:
from bs4 import BeautifulSoup as bs
import requests as req
import os
import re
from pprint import pprint
from random import randint
from time import sleep

# !pip list

In [21]:
folderPath = "project_gutenberg"
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
url = 'https://www.gutenberg.org/browse/languages/zh'
res = req.get(url)
soup = bs(res.text, "lxml")

project_gutenberg=[]
book_links = soup.select("li.pgdbetext > a[href]")
chinese = re.compile(r'[\u4e00-\u9fff]')

for book in book_links:
    title = book.get_text().strip()
    link = book.get('href')
    if chinese.search(title):
        match = re.search(r'/ebooks/(\d+)', link)
        if match:
            book_id = match.group(1)
            full_link = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt"
        
            project_gutenberg.append({
            "title": title,
            "link": full_link
            })

pprint(project_gutenberg)

In [ ]:
#檢查重複的書名
title_count = {}

for obj in project_gutenberg:
    title = re.sub(r'[\\/:*?"<>|\r\n]', '', obj["title"]).strip()
    url = obj["link"]
    res = req.get(url)
    chinese_text = re.findall(r'[\u4e00-\u9fff]+', res.text)
    content = ' '.join(chinese_text)

    if title not in title_count:
        title_count[title] = 0
        filename = f"{title}.txt"
    else:
        title_count[title] += 1
        filename = f"{title}_{title_count[title]}.txt"
    
    with open(f"{folderPath}/{filename}", "w", encoding="utf-8") as f:
        f.write(content)
        print(f"{filename}")

    sleep(randint(1, 3))